<a href="https://colab.research.google.com/github/aahftab/gemma_rag/blob/main/gemma_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import chromadb
db = chromadb.PersistentClient(path="./chroma")
collection = db.get_or_create_collection(name="hadith")

def response(query):
  results = collection.query(
      query_texts=[query],
      n_results=10
  )
  return results

In [ ]:
from openai import OpenAI

client1 = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

history = [
    {"role": "system", "content": "You are an religious assistant, which provides answers based on queries from the context of Islamic religion provided to you in the form of a query and context."}
] 

In [ ]:
def fetchResponse(query, result):
    history.append({
        "role": "user", 
        "content": f"Answer Query: {query} \n based upon the context given below \n Context: {result}"})

    completion = client1.chat.completions.create(
        model="local-model",
        messages=history, # type: ignore
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)

In [ ]:
client2 = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

In [ ]:
def summarise(text):
    completion = client2.chat.completions.create(
    model="local-model", # this field is currently unused
    messages=[
      {"role": "system", "content": "You are a very intelligent language assistant. You will provide summary in bullet points. provide me bullet points. Provide it directly without any prefixes"},
      {"role": "user", "content": f"{text}"}
    ],
    temperature=0.7,
    )
    
    return completion.choices[0].message.content
 

In [ ]:
query = "What are the basics of namaz?"
results = response(query)

In [ ]:
result = ''
count = 1
for i in results['documents'][0]: #type: ignore
  print(count)
  count += 1
  result += "-" + "-".join(summarise(i).split('-')[1:]) #type: ignore

In [ ]:
# for i in results['documents'][0]: #type: ignore
#   print(f"{summarise(i)}")
result = result.replace("\n\n\n", "\n")
result = result.replace("\n\n", "\n")
result

In [ ]:
print(f"{result}")

In [ ]:
fetchResponse(query, result)

In [ ]:
fetchResponse("Explain it more", "")